En este cuaderno se tiene el flujo completo desde el data set original hasta el data set que se lleva al entrenamiento, esto incluye varios pasos:
- Reducir las etiquetas de las imágenes (formato COCO)
- Recortar las imágenes según la clase cell phone y mantener el etiquetado original
- Aplicar SAHI a las imágenes 128x128
- Convertir de Json COCO a YOLO formato
- Dividir el conjunto de imágenes entre train, valid y test

### Reducir las etiquetas de las imágenes - Part 1

El dataset original viene dividido en varias carpetas con distintos archivos de anotación, el primer paso es fusionar todo, para trabajar con un dataset único.

In [4]:
import json

# Lista de archivos JSON a fusionar
archivos_json = ['/home/ana/TFM/datasets/original/dataset_001/annotations/instances_default.json',
                   '/home/ana/TFM/datasets/original/dataset_002/annotations/instances_default.json',
                     '/home/ana/TFM/datasets/original/dataset_003/annotations/instances_default.json',
                       '/home/ana/TFM/datasets/original/dataset_004/annotations/instances_default.json',
                         '/home/ana/TFM/datasets/original/dataset_005/annotations/instances_default.json',
                           '/home/ana/TFM/datasets/original/dataset_006/annotations/instances_default.json',
                             '/home/ana/TFM/datasets/original/dataset_007/annotations/instances_default.json',
                               '/home/ana/TFM/datasets/original/dataset_008/annotations/instances_default.json',
                                 '/home/ana/TFM/datasets/original/dataset_009/annotations/instances_default.json',
                                   '/home/ana/TFM/datasets/original/dataset_010/annotations/instances_default.json' 
                               ]

# Estructuras para almacenar la información combinada
imagenes_combinadas = []
categorias_combinadas = []
anotaciones_combinadas = []

# Último id utilizado para asegurar que 'image_id' sea único
ultimo_id = 0

# Cargar y combinar la información de cada archivo
for archivo in archivos_json:
    with open(archivo, 'r') as file:
        data = json.load(file)
        
        # Procesar cada imagen para asignar un nuevo 'image_id'
        imagenes_temporales = data.get('images', [])
        for imagen in imagenes_temporales:
            imagen_original_id = imagen['id']
            ultimo_id += 1  # Incrementar el último id utilizado
            imagen['id'] = ultimo_id  # Asignar el nuevo 'image_id'
            
            # Actualizar 'image_id' en las anotaciones correspondientes
            for anotacion in data.get('annotations', []):
                if anotacion['image_id'] == imagen_original_id:
                    anotacion['image_id'] = ultimo_id
                    anotaciones_combinadas.append(anotacion)  # Agregar la anotación actualizada

        # Agregar imágenes actualizadas
        imagenes_combinadas.extend(imagenes_temporales)

        # Agregar categorías de este archivo
        categorias_combinadas.extend(data.get('categories', []))

# Eliminar posibles duplicados en categorías si es necesario
categorias_unicas = {each['id']: each for each in categorias_combinadas}.values()

# Crear un nuevo diccionario con toda la información combinada
data_combinada = {
    'images': imagenes_combinadas,
    'categories': list(categorias_unicas),
    'annotations': anotaciones_combinadas
}

# Guardar la información combinada en un nuevo archivo JSON
with open('/home/ana/TFM/datasets/flujo_imagenes _TFM/part1_reduccion_etiquetas/annotations_reduced.json', 'w') as file:
    json.dump(data_combinada, file, indent=4)

print("Datos combinados y guardados en 'datos_combinados.json'.")


Datos combinados y guardados en 'datos_combinados.json'.


Partiendo del dataset unificado tenemos que reducir las clases

In [5]:
import json

def eliminar_clases_coco(archivo_json, clases_a_eliminar):
    with open(archivo_json, 'r') as f:
        datos = json.load(f)

    # Crear un diccionario para mapear IDs antiguos a nuevos
    id_mapping = {}
    nuevo_id = 1
    for cat in datos["categories"]:
        if cat["name"] not in clases_a_eliminar:
            id_mapping[cat["id"]] = nuevo_id
            cat["id"] = nuevo_id  # Actualizar el ID en la categoría también
            nuevo_id += 1

    # Eliminar categorías
    datos["categories"] = [cat for cat in datos["categories"] if cat["name"] not in clases_a_eliminar]

    # Eliminar anotaciones con categorías eliminadas y actualizar IDs
    datos["annotations"] = [
        ann for ann in datos["annotations"] if ann["category_id"] in id_mapping
    ]
    for ann in datos["annotations"]:
        ann["category_id"] = id_mapping[ann["category_id"]]

    # Guardar el archivo modificado
    with open(archivo_json, 'w') as f:
        json.dump(datos, f, indent=2)


archivo_coco = '/home/ana/TFM/datasets/flujo_imagenes _TFM/part1_reduccion_etiquetas/annotations_reduced.json'  # ruta del archivo con las etiquetas
clases_a_eliminar = ["Camara ara\u00f1ada", 
                     "Camara leve", 
                     "Camara rota",
                     "Chasis roto", 
                     "Falta bandeja sim",
                     "Pantalla profundo", 
                     "Pantalla rota"]  # Lista de clases a eliminar

eliminar_clases_coco(archivo_coco, clases_a_eliminar)


### Recortar las imágenes según la clase cell phone y mantener el etiquetado original - Part 2

In [6]:
import os
import json
from PIL import Image, ImageDraw, ImageFont
from pathlib import Path
from ultralytics import YOLO

# Configuración inicial
# Cargar el modelo YOLO
model = YOLO('/home/ana/TFM/modelos/pre-crop-cell-phone/weights/best.pt') #modelo YOLO customizado para detectar el telefono

# Rutas de las carpetas y archivos
input_folder = '/home/ana/TFM/datasets/flujo_imagenes _TFM/part1_reduccion_etiquetas'
output_folder = '/home/ana/TFM/datasets/flujo_imagenes _TFM/part2_crop_cell_phone'
coco_file = '/home/ana/TFM/datasets/flujo_imagenes _TFM/part1_reduccion_etiquetas/annotations_reduced.json'

output_coco = {'images': [], 'annotations': [], 'categories': []}


# Cargar las anotaciones COCO
with open(coco_file, 'r') as f:
    coco_data = json.load(f)

# Preparar mapeo de ID de categoría a nombre de categoría para COCO
category_map = {cat['id']: cat['name'] for cat in coco_data['categories']}

# Mapear anotaciones COCO por nombre de archivo
coco_annotations = {img['file_name']: [] for img in coco_data['images']}

# Añadir anotaciones a las imágenes correspondientes
for ann in coco_data['annotations']:
    image_id = ann['image_id']
    image_info = next((img for img in coco_data['images'] if img['id'] == image_id), None)
    if image_info:
        filename = image_info['file_name']
        coco_annotations[filename].append(ann)

        
# Cargar anotaciones COCO originales
with open(coco_file, 'r') as f:
    coco_data = json.load(f)
output_coco['categories'] = coco_data['categories']

# Función para ajustar anotaciones COCO
def adjust_annotations(annotations, dx, dy):
    adjusted_annotations = []
    for ann in annotations:
        new_ann = ann.copy()
        new_ann['bbox'] = [ann['bbox'][0] - dx, ann['bbox'][1] - dy, ann['bbox'][2], ann['bbox'][3]]
        adjusted_annotations.append(new_ann)
    return adjusted_annotations

# Procesar imágenes
image_id = 0
annotation_id = 0
for filename in os.listdir(input_folder):
    if filename.endswith('.png'):
        file_path = os.path.join(input_folder, filename)
        image = Image.open(file_path)
        results = model(file_path, verbose = False)
        image_saved = False
        if filename in coco_annotations:
            #print('la imagen esta')
            for result in results:
                boxes = result.boxes.cpu().numpy().xyxy
                classes = result.boxes.cpu().numpy().cls
                names = result.names
                for box, cls in zip(boxes, classes):
                    if names[cls] == "cell phone":  # Suponiendo que 'cls' contiene el nombre de la clase
                        #print('detecto cell phone')
                        crop = image.crop((box[0], box[1], box[2], box[3]))
                        crop_path = os.path.join(output_folder, f"{Path(filename).stem}.png")
                        crop.save(crop_path)
                        output_coco['images'].append({'id': image_id, 'file_name': os.path.basename(crop_path), 'width': crop.width, 'height': crop.height})
                        # Ajustar y añadir anotaciones para el recorte
                        adjusted_annotations = adjust_annotations(coco_annotations[filename], box[0], box[1])
                        for ann in adjusted_annotations:
                            ann['id'] = annotation_id
                            ann['image_id'] = image_id
                            output_coco['annotations'].append(ann)
                            annotation_id += 1
                        image_id += 1
                        image_saved = True
        if not image_saved:

            #print('entro a que la imagen no se ha recortado')
            original_path = os.path.join(output_folder, filename)
            image.save(original_path)
            output_coco['images'].append({'id': image_id, 'file_name': os.path.basename(original_path), 'width': image.width, 'height': image.height})
            try:
                #print(filename)
                #print(coco_annotations['IMG_0039_png.rf.7abe8202f5f32488df975de9519b83bc.jpg'])
                #print(f"Desde filename: {coco_annotations[filename]}")
                print(filename)
                for ann in coco_annotations[filename]:
                    new_ann = ann.copy()  # Trabaja con una copia para evitar modificar el original
                    new_ann['id'] = annotation_id
                    new_ann['image_id'] = image_id
                    output_coco['annotations'].append(new_ann)
                    annotation_id += 1
            except Exception as e:
                print(e)
            image_id += 1

# Guardar el nuevo archivo COCO
with open('/home/ana/TFM/datasets/flujo_imagenes _TFM/part2_crop_cell_phone/annotations_cropped.json', 'w') as f:
    json.dump(output_coco, f, indent=2)

print("Proceso completado.")

Proceso completado.


### Aplicar SAHI a las imágenes - Part 3

In [8]:
from sahi.slicing import slice_coco

coco_dict, coco_path = slice_coco(
    coco_annotation_file_path='/home/ana/TFM/datasets/flujo_imagenes _TFM/part2_crop_cell_phone/annotations_cropped.json',
    image_dir='/home/ana/TFM/datasets/flujo_imagenes _TFM/part2_crop_cell_phone/',
    output_coco_annotation_file_name='/home/ana/TFM/datasets/flujo_imagenes _TFM/part3_sahi/annotations_sahi.json',
    ignore_negative_samples=True,
    output_dir='/home/ana/TFM/datasets/flujo_imagenes _TFM/part3_sahi',
    slice_height=128,
    slice_width=128,
    overlap_height_ratio=0.2,
    overlap_width_ratio=0.2,
    min_area_ratio=0.1,
    verbose=False
)

indexing coco dataset annotations...


100%|██████████| 944/944 [02:40<00:00,  5.87it/s]


### Convertir de JSON COCO a YOLO

El código es externo, de un repositorio de ultralytics: https://github.com/ultralytics/JSON2YOLO, descargar este repositorio y luego hacer uso del archivo general_json2yolo.py, hay que poner como fuente "COCO", e introducir la carpeta correcta donde se encuentran las imágenes como en el siguiente ejemplo:


    if __name__ == "__main__":
    source = "COCO"

        if source == "COCO":
            convert_coco_json(
                '/home/ana/TFM/datasets/dataset_precrop_sahi_coco',  # directory with *.json
                use_segments=False,
                cls91to80=True,
            )

Esto creará en la carpeta donde se encuentre dicho archivo python una carpeta llamada new_dir, que contendrás labels e images, donde se podrá encontrar las etiquetas en formato YOLO (archivos txt con el mismo nombre que sus imágenes asociadas).

In [9]:
import os
import shutil

def mover_contenido(directorio_origen, directorio_destino):
    # Lista todos los archivos y directorios en el directorio origen
    elementos = os.listdir(directorio_origen)
    
    for elemento in elementos:
        # Construye la ruta completa del elemento en el directorio origen
        ruta_completa_origen = os.path.join(directorio_origen, elemento)
        
        # Mueve cada elemento al directorio destino
        shutil.move(ruta_completa_origen, directorio_destino)

# Rutas de los directorios
directorio_origen = '/home/ana/JSON2YOLO/new_dir/labels'
directorio_destino = '/home/ana/TFM/datasets/flujo_imagenes _TFM/part4_yolo_format'

# Llama a la función para mover el contenido
mover_contenido(directorio_origen, directorio_destino)


### Dividir el conjunto de imágenes (train/valid/test) - Part 5

In [17]:
import os
import shutil
from random import shuffle
import yaml
from collections import OrderedDict


# Rutas a las carpetas donde se encuentran todas las imágenes y etiquetas
carpeta_imagenes = '/home/ana/TFM/datasets/flujo_imagenes _TFM/part3_sahi'
carpeta_labels = '/home/ana/TFM/datasets/flujo_imagenes _TFM/part4_yolo_format/annotations_sahi.json_coco'

# Crea las carpetas para los conjuntos de datos
sets = ['train', 'valid', 'test']
for set_name in sets:
    os.makedirs(f'/home/ana/TFM/datasets/flujo_imagenes _TFM/part5_split/{set_name}/images', exist_ok=True)
    os.makedirs(f'/home/ana/TFM/datasets/flujo_imagenes _TFM/part5_split/{set_name}/labels', exist_ok=True)

# Lista todas las imágenes y etiquetas
all_images = [f for f in os.listdir(carpeta_imagenes) if f.endswith(('.jpg', '.jpeg', '.png'))]
all_labels = [f for f in os.listdir(carpeta_labels) if f.endswith('.txt')]

# Asegúrate de que cada imagen tenga su correspondiente archivo de etiqueta
all_files = [(img, img.replace('.png', '.txt')) for img in all_images if img.replace('.png', '.txt') in all_labels]

# Baraja aleatoriamente los archivos
shuffle(all_files)

# Asigna archivos a cada conjunto
train_files = all_files[:10000]
valid_files = all_files[10000:12000]
test_files = all_files[12000:14000]

# Función para mover los archivos
def move_files(files, set_name):
    for img, label in files:
        # Mover imágenes
        shutil.move(os.path.join(carpeta_imagenes, img), f'/home/ana/TFM/datasets/flujo_imagenes _TFM/part5_split/{set_name}/images/{img}')
        # Mover etiquetas
        shutil.move(os.path.join(carpeta_labels, label), f'/home/ana/TFM/datasets/flujo_imagenes _TFM/part5_split/{set_name}/labels/{label}')

# Mueve los archivos a las carpetas correspondientes
move_files(train_files, 'train')
move_files(valid_files, 'valid')
move_files(test_files, 'test')

print("Los archivos han sido distribuidos correctamente en las carpetas de train, valid y test.")

Los archivos han sido distribuidos correctamente en las carpetas de train, valid y test.
